# Analysis of Airbnb data on multiple locations spread across Spain

---


#### Setup


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys, os
import logging
import pandas as pd
import seaborn as sns
import json
import numpy as np

from copy import deepcopy
import datetime
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.metrics import r2_score
from sklearn.model_selection import KFold
from pathlib import Path
from typing import Iterable
from IPython import display as ICD
from matplotlib import pyplot as plt

In [ ]:
src_path: str = "../src"
sys.path.append(src_path)
logging.getLogger().setLevel(logging.WARN)

In [ ]:
from data_wrangling import *

In [ ]:
random_seed: int = 8080

---


In [ ]:
pd.read_csv("../data/airbnb/euskadi/calendar.csv.bz2", index_col=0)

In [ ]:
pd.read_csv("../data/airbnb/euskadi/listings.csv.bz2", index_col=0)

## 1. Data exploration

In this section I will be exploring the Airbnb data schema. I will choose Madrid for this task, but all locations follow the same schema.


In [ ]:
madrid_files: Path = Path("../data/airbnb/madrid")

In [ ]:
listings_schema_df = pd.read_csv(
    madrid_files.parent.joinpath("listings_schema.csv")
).set_index("Field")
listings_df = pd.read_csv(madrid_files.joinpath("listings.csv.bz2"))
calendar_df = pd.read_csv(madrid_files.joinpath("calendar.csv.bz2"))

### 1.1. Listings


Listings fields with descriptions


In [ ]:
listings_with_desc = listings_schema_df[["Description"]].dropna()
listings_with_desc

Listings fields without descriptions


In [ ]:
listings_without_desc = listings_schema_df[["Description"]][
    listings_schema_df["Description"].isna()
]
listings_without_desc.index.tolist()

### 1.2. Calendar


In [ ]:
calendar_df.head()

## 2. Data wrangling

In this section we will be massaging the data to answer our business questions.


In [ ]:
airbnb_files: Path = Path("../data/airbnb")

### 2.1. _What is the average price of each location type per neighbourhood? What are the most expensive neighbourhoods on average?_


In [ ]:
most_expensive_hoods = {}
for path in airbnb_files.glob("**/listings.csv.bz2"):
    region_name = path.parent.name
    most_expensive_hoods[region_name.title()] = airbnb_avg_price(path)[1]

most_expensive_hoods_df = pd.DataFrame(most_expensive_hoods).transpose()
most_expensive_hoods_df.to_csv(Path("../data/results/question_1.csv"))
most_expensive_hoods_df

### 2.2. _What is the average host acceptance rate per location type and neighborhood? In which neighbourhoods is it the highest and in which the lowest?_

This can give us an idea of the negotiating power of the hosts or the desirability of guests.


In [ ]:
highest_accept_rate_hoods = {}
for path in airbnb_files.glob("**/listings.csv.bz2"):
    region_name = path.parent.name
    highest_accept_rate_hoods[region_name.title()] = airbnb_avg_accept_rate(path)[1]

highest_accept_rate_hoods_df = pd.DataFrame(highest_accept_rate_hoods).transpose()
highest_accept_rate_hoods_df.to_csv(Path("../data/results/question_2.csv"))
highest_accept_rate_hoods_df

### 2.3. _What number and proportion of listings per neighbourhood belong to hosts owning different numbers of locations? In which neighbourhoods is the concentration bigger?_


In [ ]:
most_dense_hoods = {}
for path in airbnb_files.glob("**/listings.csv.bz2"):
    region_name = path.parent.name
    most_dense_hoods[region_name.title()] = airbnb_hood_hosts(path)[1]

most_dense_hoods_df = pd.DataFrame(most_dense_hoods).transpose()
most_dense_hoods_df.to_csv(Path("../data/results/question_3.csv"))
most_dense_hoods_df

### 2.4. _What is the expected average profit per room type and neighborhood when looking at the reservations for the next N weeks? What is the neighbourhood expected to be the most profitable in that period?_

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


In [ ]:
n_weeks = 6 * 4

In [ ]:
most_profitable_hoods = {}
for listings_path, calendar_path in zip(
    sorted(airbnb_files.glob("**/listings.csv.bz2")),
    sorted(airbnb_files.glob("**/calendar.csv.bz2")),
):
    region_name = listings_path.parent.name
    most_profitable_hoods[region_name.title()] = airbnb_avg_profit(
        listings_path, calendar_path, n_weeks=n_weeks
    )[1]

most_profitable_hoods_df = pd.DataFrame(most_profitable_hoods).transpose()
most_profitable_hoods_df.to_csv(Path("../data/results/question_4.csv"))
most_profitable_hoods_df

### 2.5. _What listings' factors affect the total profit in the next N weeks? Can the total profit be predicted?_

Here we assume that none of the reserved dates will be cancelled and that they are a good representation of the observed period.


In [ ]:
profit_predictions = {}
for listings_path, calendar_path in zip(
    sorted(airbnb_files.glob("**/listings.csv.bz2")),
    sorted(airbnb_files.glob("**/calendar.csv.bz2")),
):
    region_name = listings_path.parent.name
    num_features, cat_features, mean_r2_score = airbnb_predict_profit(
        listings_path,
        calendar_path,
        feature_th=0,
        n_weeks=n_weeks,
        random_seed=random_seed,
    )
    profit_predictions[region_name.title()] = {
        "num_features": len(num_features),
        "cat_features": len(cat_features),
        "mean_r2_score": mean_r2_score,
    }

profit_predictions_df = pd.DataFrame(profit_predictions).transpose()
profit_predictions_df.to_csv(Path("../data/results/question_5.csv"))
profit_predictions_df